In [2]:
import pandas as pd

df = pd.read_csv('datos/test.csv')

In [6]:
import string
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords

def preprocesar(c):

    wnl = WordNetLemmatizer()
    ps = PorterStemmer()
    remove = []
    for i in range(len(c)):
        s = str(c[i])
        if not bool(s):
            remove.append(df['id'])
        else:
            s = s.lower()
            s = s.translate(str.maketrans('', '', string.punctuation))
            s = s.translate(str.maketrans('', '', string.digits))
            s = s.split()
            stop_words=set(stopwords.words('english'))
            filtered_sentence = [w for w in s if not w in stop_words]
            filtered_sentence = []
            for w in s:
                if w not in stop_words and len(w)>1 and w.isascii():
                    w = wnl.lemmatize(w)
                    w = ps.stem(w)
                    if len(w)>1:
                        filtered_sentence.append(w)
            if not bool(filtered_sentence):
                remove.append(i)
            c[i] = filtered_sentence
    return(c, remove)    
    

In [12]:
df['title'], r1 = preprocesar(df['title'])

/tmp/ipykernel_3884/346490496.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c[i] = filtered_sentence


In [13]:
df['text'], r2 = preprocesar(df['text'])

/tmp/ipykernel_3884/346490496.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c[i] = filtered_sentence


In [14]:
r = r1+r2
r = list(dict.fromkeys(r))
df = df.drop(df.index[r])

In [15]:
df.to_csv('datos prep/test_p.csv', index=False)

In [7]:
df = pd.read_csv('datos prep/test_p.csv')
df = df.drop(columns=['title', 'id', 'author'])
df['id'] = range(1, len(df) + 1)
print(df)

                                                   text    id
0     ['palo', 'alto', 'calif', 'year', 'scorn', 'po...     1
1     ['russian', 'warship', 'readi', 'strike', 'ter...     2
2     ['video', 'nodapl', 'nativ', 'american', 'lead...     3
3     ['first', 'succeed', 'tri', 'differ', 'sport',...     4
4     ['min', 'ago', 'view', 'comment', 'like', 'fir...     5
...                                                 ...   ...
5116  ['dysfunct', 'plagu', 'megac', 'none', 'may', ...  5117
5117  ['washington', 'gov', 'john', 'kasich', 'ohio'...  5118
5118  ['good', 'morn', 'want', 'get', 'california', ...  5119
5119  ['previou', 'next', 'marin', 'deploy', 'russia...  5120
5120  ['perhap', 'seen', 'new', 'tv', 'seri', 'whose...  5121

[5121 rows x 2 columns]


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvectorizer = TfidfVectorizer(analyzer='word')
tfidf_wm = tfidfvectorizer.fit_transform(df['text'])
n=tfidf_wm.shape[0]+1
l = [item for item in range(1, n+1)]
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray())
i=1
for r in df_tfidfvect:
    df_tfidfvect[r].insert(0, l[i])
    i=i+1



print(df_tfidfvect)

AttributeError: 'Series' object has no attribute 'insert'

In [1]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd


df = pd.read_csv('datos prep/test_p.csv')
df = df[:3000]
tfidfvectorizer = TfidfVectorizer(analyzer='word')
tfidf_wm = tfidfvectorizer.fit_transform(df['text'])
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray())
n=tfidf_wm.shape[0]
l = [item for item in range(1, n+1)]
df_tfidfvect.insert(0, "id", l)
arr = df_tfidfvect.values.tolist()
with open('datos prep/test_tfidf100.csv', 'w') as f:
    write = csv.writer(f)
    write.writerows(arr)
#arr.to_csv('datos prep/test_tfidf100.csv', index=False)